In [2]:
import os
import json

import octomap
import numpy as np
import open3d as o3d

In [3]:
DATA_ROOT = "../../raw/jrdb"
LABELS_DIR = "labels/labels_3d"
PCD_DIR = "pointclouds/lower_velodyne"

## Explore labels

In [4]:
label_files = [filename for filename in os.listdir(os.path.join(DATA_ROOT, LABELS_DIR)) if filename.endswith(".json")]

In [5]:
locations = set([filename.split("_")[0] for filename in label_files])

In [6]:
locations

{'bytes-cafe-2019-02-07',
 'clark-center-2019-02-28',
 'clark-center-intersection-2019-02-28',
 'cubberly-auditorium-2019-04-22',
 'forbes-cafe-2019-01-22',
 'gates-159-group-meeting-2019-04-03',
 'gates-ai-lab-2019-02-08',
 'gates-basement-elevators-2019-01-17',
 'gates-to-clark-2019-02-28',
 'hewlett-packard-intersection-2019-01-24',
 'huang-2-2019-01-25',
 'huang-basement-2019-01-25',
 'huang-lane-2019-02-12',
 'jordan-hall-2019-04-22',
 'memorial-court-2019-03-16',
 'meyer-green-2019-03-16',
 'nvidia-aud-2019-04-18',
 'packard-poster-session-2019-03-20',
 'stlc-111-2019-04-19',
 'svl-meeting-gates-2-2019-04-08',
 'tressider-2019-03-16',
 'tressider-2019-04-26'}

In [7]:
# Print all different locations in this dataset
print(f"There are {len(locations)} different locations in this dataset:")
print(locations)

There are 22 different locations in this dataset:
{'gates-159-group-meeting-2019-04-03', 'jordan-hall-2019-04-22', 'cubberly-auditorium-2019-04-22', 'meyer-green-2019-03-16', 'hewlett-packard-intersection-2019-01-24', 'clark-center-intersection-2019-02-28', 'huang-2-2019-01-25', 'memorial-court-2019-03-16', 'forbes-cafe-2019-01-22', 'huang-lane-2019-02-12', 'clark-center-2019-02-28', 'gates-ai-lab-2019-02-08', 'stlc-111-2019-04-19', 'gates-to-clark-2019-02-28', 'nvidia-aud-2019-04-18', 'tressider-2019-04-26', 'tressider-2019-03-16', 'svl-meeting-gates-2-2019-04-08', 'huang-basement-2019-01-25', 'gates-basement-elevators-2019-01-17', 'bytes-cafe-2019-02-07', 'packard-poster-session-2019-03-20'}


In [8]:
with open(os.path.join(DATA_ROOT, LABELS_DIR, label_files[0]), "r") as fp:
    label = json.load(fp)['labels']

In [9]:
label.keys()

dict_keys(['000000.pcd', '000001.pcd', '000002.pcd', '000003.pcd', '000004.pcd', '000005.pcd', '000006.pcd', '000007.pcd', '000008.pcd', '000009.pcd', '000010.pcd', '000011.pcd', '000012.pcd', '000013.pcd', '000014.pcd', '000015.pcd', '000016.pcd', '000017.pcd', '000018.pcd', '000019.pcd', '000020.pcd', '000021.pcd', '000022.pcd', '000023.pcd', '000024.pcd', '000025.pcd', '000026.pcd', '000027.pcd', '000028.pcd', '000029.pcd', '000030.pcd', '000031.pcd', '000032.pcd', '000033.pcd', '000034.pcd', '000035.pcd', '000036.pcd', '000037.pcd', '000038.pcd', '000039.pcd', '000040.pcd', '000041.pcd', '000042.pcd', '000043.pcd', '000044.pcd', '000045.pcd', '000046.pcd', '000047.pcd', '000048.pcd', '000049.pcd', '000050.pcd', '000051.pcd', '000052.pcd', '000053.pcd', '000054.pcd', '000055.pcd', '000056.pcd', '000057.pcd', '000058.pcd', '000059.pcd', '000060.pcd', '000061.pcd', '000062.pcd', '000063.pcd', '000064.pcd', '000065.pcd', '000066.pcd', '000067.pcd', '000068.pcd', '000069.pcd', '000070.p

In [10]:
label['000000.pcd'][0].keys()

dict_keys(['attributes', 'box', 'file_id', 'label_id', 'observation_angle'])

In [11]:
# So for each PCD file, we have a list of boxes (which are labels on that frame)
tmp = label['000000.pcd'][0]["box"]
cx, cy, cz, w, h, l, rot_z = tmp["cx"], tmp["cy"], tmp["cz"], tmp["w"], tmp["h"], tmp["l"], tmp["rot_z"]
print(f"Example of a label: cx={cx}, cy={cy}, cz={cz}, w={w}, h={h}, l={l}, rot_z={rot_z}")

Example of a label: cx=-1.42, cy=-23.13, cz=0.435, w=0.53, h=1.79, l=0.73, rot_z=-5.38696


## Explore pointclouds

In [12]:
label_files[23]

'bytes-cafe-2019-02-07_0.json'

In [13]:
PCD_FILE = 'bytes-cafe-2019-02-07_0'

In [14]:
pcd = o3d.io.read_point_cloud(os.path.join(DATA_ROOT, PCD_DIR, PCD_FILE, "000000.pcd"))
pcd = np.asarray(pcd.points)

In [15]:
# Union PCDS
pcd_root = os.path.join(DATA_ROOT, PCD_DIR, PCD_FILE)
pcd_files = [filename for filename in os.listdir(pcd_root) if filename.endswith(".pcd")]

pcds = []
idx = 0
for pcd_file in pcd_files:
    idx += 1
    if idx % 20 == 1:
        pcd = o3d.io.read_point_cloud(os.path.join(pcd_root, pcd_file))
        pcds.append(np.asarray(pcd.points))
pcd = np.concatenate(pcds, axis=0)
print(pcd.shape)
o3d.io.write_point_cloud("test.pcd", o3d.geometry.PointCloud(o3d.utility.Vector3dVector(pcd)))

(1506762, 3)


True

In [16]:
octree = octomap.OcTree(0.1)
octree.insertPointCloud(
        pointcloud=pcds[0],
        origin=np.array([0,0,0], dtype=float),
        maxrange=2,
)

In [18]:
occupied, empty = octree.extractPointCloud()

AttributeError: 'octomap.leaf_iterator' object has no attribute '_leaf_iterator__is_end'